In [112]:
pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=603465 sha256=7ee4f9546c6e35306b2fa860ca519460bbea282c2f6f97d40e9ffdf8632313a1
  Stored in directory: /home/ubuntu/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy
Note: you may need to restart the kernel to use updated packages.


In [104]:
import pickle
import pandas as pd
from tqdm import tqdm
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

In [98]:
class Paper:
    def __init__(self, url, title, author, published, refs):
        self.url = url
        self.title = title
        self.author = author
        self.published = published
        self.refs = refs
    
with open('/home/ubuntu/Documents/PSP/dump/papers_stucture_recommender.pickle', 'rb') as handle:
    papers = pickle.load(handle)[:200]

In [99]:

m = csr_matrix((len(papers), len(papers)), dtype=np.int8)

In [100]:
def similar(title, refs):
    title = set(title.split())
    for ref in refs:
        if len(title & set(ref.split())) / len(title) > 0.5:
            return True
    return False

In [101]:
for i, p1 in (enumerate(tqdm(papers))):
    for j, p2 in enumerate(papers):
        if similar(p1.title, p2.refs):
            # df.loc[i, j] = 1
            m[i, j] = 1
            m[j, i] = 1


100%|██████████| 200/200 [00:20<00:00,  9.70it/s]


In [81]:
m.sum().sum()

np.int64(47)

In [107]:
rows = (m.sum(axis=0) > 0)[0]
indices = rows.nonzero()[1]

In [108]:
m_good = m[indices].T[indices].toarray()

In [91]:
# df_ = df.drop(df[df.sum() == 0].index, axis=1)
# df_[df_.sum(axis=1) > 0]

In [93]:
m_good

array([[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

In [110]:
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)

m_new = svd.fit_transform(m_good)

In [111]:
m_new.shape

(75, 5)

In [114]:
from annoy import AnnoyIndex
import random

f = 5  # Length of item vector that will be indexed

t = AnnoyIndex(f, 'angular')
for i, v in enumerate((m_new)):
    t.add_item(i, v)

t.build(10) # 10 trees


True

In [ ]:
# t.save('test.ann')

# ...

u = AnnoyIndex(f, 'angular')
u.load('test.ann') # super fast, will just mmap the file
print(u.get_nns_by_item(0, 1000)) # will find the 1000 nearest neighbors

In [121]:
src_index = 17
print(papers[src_index].title)
print('='*50)
for i in t.get_nns_by_item(src_index, 5):
    print(papers[i].title)

A Survey on the Fairness of Recommender Systems
A Survey on the Fairness of Recommender Systems
Formalizing Multimedia Recommendation through Multimodal Deep Learning
Deviation-Based Learning: Training Recommender Systems Using Informed User Choice
Conformal Group Recommender System
FairMatch: A Graph-based Approach for Improving Aggregate Diversity in Recommender Systems
